In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint

In [2]:
# Create the image.ImageDataGenerator
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.8,
)

In [3]:
# Create training set as Numpy array
(x_train, y_train) = next(img_gen.flow_from_directory(
    "./PetImages",
    target_size=(224,224),
    batch_size=32,
    shuffle=True,
    subset='training'
))

Found 5000 images belonging to 2 classes.


In [4]:
# Create validation set as Numpy array
(x_val, y_val) = next(img_gen.flow_from_directory(
    "./PetImages",
    target_size=(224,224),
    batch_size=32,
    shuffle=True,
    subset='validation'
))

Found 20000 images belonging to 2 classes.


In [5]:
# Build model by creating instance of VGG16
model = keras.applications.VGG16(
    include_top=True,
    weights=None,
    classes=2,
    classifier_activation='softmax'
)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-04-08 16:34:24.406410: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-08 16:34:24.406521: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# Print model details
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
# Specify optimizer, loss, and metrics for model
model.compile(
    optimizer='rmsprop', 
    loss='categorical_crossentropy'
)

In [8]:
# Specify callbacks
model_checkpoint_callback = ModelCheckpoint(
    "./models/checkpoints/save_at_epoch_{epoch}.h5",
    monitor="val_loss",
    save_best_only=True,
)

In [9]:
# Fit data to model
history = model.fit(
    x_train, 
    y_train, 
    batch_size=32, 
    epochs=1,
    callbacks=[model_checkpoint_callback],
    validation_data=(x_val, y_val)
)

2022-04-08 16:34:24.670507: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-04-08 16:34:25.216261: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.6941

2022-04-08 16:34:28.146175: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 5s 5s/step - loss: 0.6941 - val_loss: 48818216.0000


In [10]:
# View training loss and metrics
print(history.history)

{'loss': [0.6940956711769104], 'val_loss': [48818216.0]}


In [11]:
# Save state of model after training
# Disabled for now due to implementation of callbacks
#model.save('./models/model')